In [2]:
import xarray as xr
# import xskillscore
import numpy as np
from scipy import stats
import glob
from IPython.display import clear_output
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [3]:
time_slice = slice('1/1/1970', '12/31/2020')

In [4]:
model_list =  [
    'ACCESS-CM2',
    'ACCESS-ESM1-5', 
    'BCC-CSM2-MR',
    'CanESM5', 
    'CNRM-CM6-1',
    'HadGEM3-GC31-LL', 
    'IPSL-CM6A-LR', 
    'MIROC6', 
    'MRI-ESM2-0', 
    'NorESM2-LM'
]

In [5]:
experiment_list = ['ssp_extended', 'hist-nat', 'hist-GHG']

In [6]:
analysis_period = slice('1970-01-01', '2020-12-31')

In [11]:
cmap = 'bwr'
norm = colors.TwoSlopeNorm(vmin=-0.8, vcenter=0, vmax=0.8)

# Tmax

In [10]:
for model in model_list:
    fig, axs = plt.subplots(ncols=3, figsize=(18, 6), subplot_kw={'projection': ccrs.PlateCarree()})

    for axi, experiment in enumerate(experiment_list):
        tmax_da = xr.open_dataarray(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model}_{experiment}_koreaTmax.nc').sel(time=analysis_period)
        JA_tmax = tmax_da.where(tmax_da.time.dt.month.isin([7,8])).dropna('time').resample(time='1Y').mean()
        rlut_da = xr.open_dataarray(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model}_{experiment}_rlut.nc').sel(time=analysis_period)
        JA_rlut = rlut_da.where(rlut_da.time.dt.month.isin([7,8])).dropna('time').resample(time='1Y').mean()
        
        pearson_r = xr.corr(JA_rlut, JA_tmax, dim='time')
        dof = JA_rlut.time.size - 2
        alpha_95 = 0.05
        tc_95 = stats.t.ppf(1 - alpha_95/2, dof)
        rc_95 = tc_95/np.sqrt(dof+tc_95**2)
        alpha_99 = 0.01
        tc_99 = stats.t.ppf(1 - alpha_99/2, dof)
        rc_99 = tc_99/np.sqrt(dof+tc_99**2)

        ax = axs[axi]
        c = ax.pcolormesh(pearson_r.lon, pearson_r.lat, pearson_r, transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
        
        sig_95 = (abs(pearson_r)>rc_95)
        sig_99 = (abs(pearson_r)>rc_99)
        lon_mesh, lat_mesh = np.meshgrid(pearson_r.lon, pearson_r.lat)
        ax.scatter(lon_mesh[np.where(sig_95)], lat_mesh[np.where(sig_95)], marker='.', s=1, color='green', transform=ccrs.PlateCarree())
        ax.scatter(lon_mesh[np.where(sig_99)], lat_mesh[np.where(sig_99)], marker='.', s=4, color='green', transform=ccrs.PlateCarree())
        
        ax.add_feature(cfeature.COASTLINE)
        ax.set_extent([30, 180, 0, 60], crs=ccrs.PlateCarree())
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='grey', alpha=0.8, linestyle='-')
        gl.bottom_labels = True
        gl.right_labels = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'rotation': 45}
        
        ax.set_title(experiment)
    fig.suptitle(model, y=0.75)
    cb = fig.colorbar(c, ax=axs, shrink=0.8, orientation='horizontal')
    cb.ax.set_title('JA $T_{max}$ and OLR Correlation')
    plt.savefig(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/JA_Tmax_OLR_Corr_Maps/{model}_comparison.png', bbox_inches='tight')
    plt.close()

# Num HWs

In [14]:
for model in model_list:
    fig, axs = plt.subplots(ncols=3, figsize=(18, 6), subplot_kw={'projection': ccrs.PlateCarree()})

    for axi, experiment in enumerate(experiment_list):
        tmax_da = xr.open_dataarray(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model}_{experiment}_numKoreaHWs.nc').sel(year=analysis_period)
        JA_tmax = tmax_da.rename(year='time').resample(time='1Y').mean() #tmax_da.where(tmax_da.time.dt.month.isin([7,8])).dropna('time').resample(time='1Y').mean()
        rlut_da = xr.open_dataarray(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model}_{experiment}_rlut.nc').sel(time=analysis_period)
        JA_rlut = rlut_da.where(rlut_da.time.dt.month.isin([7,8])).dropna('time').resample(time='1Y').mean()
        
        pearson_r = xr.corr(JA_rlut, JA_tmax, dim='time')
        dof = JA_rlut.time.size - 2
        alpha_95 = 0.05
        tc_95 = stats.t.ppf(1 - alpha_95/2, dof)
        rc_95 = tc_95/np.sqrt(dof+tc_95**2)
        alpha_99 = 0.01
        tc_99 = stats.t.ppf(1 - alpha_99/2, dof)
        rc_99 = tc_99/np.sqrt(dof+tc_99**2)

        ax = axs[axi]
        c = ax.pcolormesh(pearson_r.lon, pearson_r.lat, pearson_r, transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
        
        sig_95 = (abs(pearson_r)>rc_95)
        sig_99 = (abs(pearson_r)>rc_99)
        lon_mesh, lat_mesh = np.meshgrid(pearson_r.lon, pearson_r.lat)
        ax.scatter(lon_mesh[np.where(sig_95)], lat_mesh[np.where(sig_95)], marker='.', s=1, color='green', transform=ccrs.PlateCarree())
        ax.scatter(lon_mesh[np.where(sig_99)], lat_mesh[np.where(sig_99)], marker='.', s=4, color='green', transform=ccrs.PlateCarree())
        
        ax.add_feature(cfeature.COASTLINE)
        ax.set_extent([30, 180, 0, 60], crs=ccrs.PlateCarree())
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='grey', alpha=0.8, linestyle='-')
        gl.bottom_labels = True
        gl.right_labels = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'rotation': 45}
        
        ax.set_title(experiment)
    fig.suptitle(model, y=0.8)
    cb = fig.colorbar(c, ax=axs, shrink=0.8, orientation='horizontal')
    cb.ax.set_title('JA $N_{HW}$ and OLR Correlation')
    
    plt.savefig(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/JA_NHW_OLR_Corr_Maps/{model}_comparison.png', bbox_inches='tight')
    plt.close()

# Num HW Days

In [13]:
for model in model_list:
    fig, axs = plt.subplots(ncols=3, figsize=(18, 6), subplot_kw={'projection': ccrs.PlateCarree()})

    for axi, experiment in enumerate(experiment_list):
        tmax_da = xr.open_dataarray(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model}_{experiment}_Korea_HW_days.nc').sel(year=analysis_period)
        JA_tmax = tmax_da.rename(year='time').resample(time='1Y').mean() #tmax_da.where(tmax_da.time.dt.month.isin([7,8])).dropna('time').resample(time='1Y').mean()
        rlut_da = xr.open_dataarray(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model}_{experiment}_rlut.nc').sel(time=analysis_period)
        JA_rlut = rlut_da.where(rlut_da.time.dt.month.isin([7,8])).dropna('time').resample(time='1Y').mean()
        
        pearson_r = xr.corr(JA_rlut, JA_tmax, dim='time')
        dof = JA_rlut.time.size - 2
        alpha_95 = 0.05
        tc_95 = stats.t.ppf(1 - alpha_95/2, dof)
        rc_95 = tc_95/np.sqrt(dof+tc_95**2)
        alpha_99 = 0.01
        tc_99 = stats.t.ppf(1 - alpha_99/2, dof)
        rc_99 = tc_99/np.sqrt(dof+tc_99**2)

        ax = axs[axi]
        c = ax.pcolormesh(pearson_r.lon, pearson_r.lat, pearson_r, transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
        
        sig_95 = (abs(pearson_r)>rc_95)
        sig_99 = (abs(pearson_r)>rc_99)
        lon_mesh, lat_mesh = np.meshgrid(pearson_r.lon, pearson_r.lat)
        ax.scatter(lon_mesh[np.where(sig_95)], lat_mesh[np.where(sig_95)], marker='.', s=1, color='green', transform=ccrs.PlateCarree())
        ax.scatter(lon_mesh[np.where(sig_99)], lat_mesh[np.where(sig_99)], marker='.', s=4, color='green', transform=ccrs.PlateCarree())
        
        ax.add_feature(cfeature.COASTLINE)
        ax.set_extent([30, 180, 0, 60], crs=ccrs.PlateCarree())
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='grey', alpha=0.8, linestyle='-')
        gl.bottom_labels = True
        gl.right_labels = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'rotation': 45}
        
        ax.set_title(experiment)
    fig.suptitle(model, y=0.8)
    cb = fig.colorbar(c, ax=axs, shrink=0.8, orientation='horizontal')
    cb.ax.set_title('JA No. HW Days and OLR Correlation')
    
    plt.savefig(f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/JA_NumHWDays_OLR_Corr_Maps/{model}_comparison.png', bbox_inches='tight')
    plt.close()